In [1]:
import pandas as pd

import os

In [2]:
# df = pd.read_csv('Abdullah.csv')

In [3]:
# df

,data.timestamp,data.number,data.application,data.id,audioname,chunkname,data.rawTranscriptions
0,2021-10-15T18:34:03.467+0000,3019045938,ARI6,1.634323e+09,ARI6-1634322843.4127-in.wav,ARI6-1634322843.4127-in-1.wav,right back you
1,2021-10-15T18:34:03.467+0000,3019045938,ARI6,1.634323e+09,ARI6-1634322843.4127-in.wav,ARI6-1634322843.4127-in-2.wav,if you are satisfied with your message
2,2021-10-15T18:34:03.782+0000,6067847385,ARI8,1.634323e+09,ARI8-1634322843.26668-in.wav,ARI8-1634322843.26668-in-1.wav,yeah
3,2021-10-15T18:34:03.782+0000,6067847385,ARI8,1.634323e+09,ARI8-1634322843.26668-in.wav,ARI8-1634322843.26668-in-2.wav,no
4,2021-10-15T18:34:03.782+0000,6067847385,ARI8,1.634323e+09,ARI8-1634322843.26668-in.wav,ARI8-1634322843.26668-in-3.wav,let me take you something are you listening
...,...,...,...,...,...,...,...
672701,2021-10-23T16:45:37.823+0000,8165234830,ARI7,1.635008e+09,ARI7-1635007537.1644-in.wav,ARI7-1635007537.1644-in-1.wav,hello
672702,2021-10-23T16:45:37.823+0000,8165234830,ARI7,1.635008e+09,ARI7-1635007537.1644-in.wav,ARI7-1635007537.1644-in-2.wav,no you please i do not
672703,2021-10-23T16:45:38.053+0000,8646544741,ARI7,1.635008e+09,ARI7-1635007538.16449-in.wav,ARI7-1635007538.16449-in-1.wav,come bye
672704,2021-10-23T16:45:38.152+0000,9287724043,ARI7,1.635008e+09,ARI7-1635007538.1645-in.wav,ARI7-1635007538.1645-in-1.wav,hello


In [2]:
audio_file_name_list = os.listdir(r"Z:\audios\T_1")

In [3]:
len(audio_file_name_list)

103445

In [15]:
# df['audioname'].unique()

array(['ARI6-1634322843.4127-in.wav', 'ARI8-1634322843.26668-in.wav',
       'ARI10-1634322844.3208802-in.wav', ...,
       'ARI7-1635007537.1644-in.wav', 'ARI7-1635007538.16449-in.wav',
       'ARI7-1635007538.1645-in.wav'], dtype=object)

In [16]:
# len(df['audioname'].unique())

373601

In [17]:
# df_file_name_list = df['audioname'].unique().tolist()

In [18]:
# len(df_file_name_list)

373601

In [19]:
# file_names =  set(df_file_name_list) & set(audio_file_name_list)

In [20]:
# len(file_names)

33283

In [4]:
# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.

from pydub import AudioSegment
from pydub.silence import split_on_silence

# Define a function to normalize a chunk to a target amplitude.

def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

count=0

C:\Users\trainee6\Anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [5]:
# previous chunks 4097

# import os

# file_names = os.listdir(r"Z:\audios\T_1")

try :
    
    for file_name in audio_file_name_list:
        # Load your audio.
        song = AudioSegment.from_wav("Z:/audios/T_1/" + str(file_name))

        # Split track where the silence is 2 seconds or more and get chunks using 
        # the imported function.
        chunks = split_on_silence (
            # Use the loaded audio.
            song, 
            # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
            min_silence_len = 1800, #1800
            # Consider a chunk silent if it's quieter than -16 dBFS.
            # (You may want to adjust this parameter.)
            silence_thresh = song.dBFS, #-27

            keep_silence=300,

            seek_step=1
        )

        file_name = file_name.replace('.wav', '')
        
        print('Audio File # ' ,count)
        count+=1
        
        # Process each chunk with your parameters
        for i, chunk in enumerate(chunks):
            # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
            silence_chunk = AudioSegment.silent(duration=500)

            # Add the padding chunk to beginning and end of the entire chunk.
            audio_chunk = silence_chunk + chunk + silence_chunk

            # Normalize the entire chunk.
            normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

            # Export the audio chunk with new bitrate.
            print("Exporting" + file_name + "-{0}.wav".format(i))
            normalized_chunk.export(
                "audio_chunks_T_1/" + file_name + "-{0}.wav".format(i+1),
                bitrate = "192k",
                format = "wav"
            )
            
            
except Exception as e:
        print(e)
            
            
# except FileNotFoundError:
#     pass

# except RuntimeWarning:
#     pass

Audio File #  0
ExportingARI6-1635004895.13788-out-0.wav
ExportingARI6-1635004895.13788-out-1.wav
Audio File #  1
ExportingARI5-1635886138.7133-in-0.wav
Audio File #  2
ExportingARI11-1635868694.5812-in-0.wav
Audio File #  3
ExportingARI9-1635957483.12531-in-0.wav
Audio File #  4
ExportingARI9-1635778967.4802-in-0.wav
ExportingARI9-1635778967.4802-in-1.wav
ExportingARI9-1635778967.4802-in-2.wav
Audio File #  5
ExportingARI8-1635807597.25953-in-0.wav
ExportingARI8-1635807597.25953-in-1.wav
ExportingARI8-1635807597.25953-in-2.wav
ExportingARI8-1635807597.25953-in-3.wav
Audio File #  6
ExportingARI12-1635870626.16083-in-0.wav
ExportingARI12-1635870626.16083-in-1.wav
Audio File #  7
ExportingARI5-1635892233.20086-in-0.wav
ExportingARI5-1635892233.20086-in-1.wav
Audio File #  8
ExportingARI9-1635787215.44734-in-0.wav
ExportingARI9-1635787215.44734-in-1.wav
ExportingARI9-1635787215.44734-in-2.wav
ExportingARI9-1635787215.44734-in-3.wav
ExportingARI9-1635787215.44734-in-4.wav
Audio File #  9


Audio File #  86
ExportingARI6-1634932302.467-out-0.wav
ExportingARI6-1634932302.467-out-1.wav
ExportingARI6-1634932302.467-out-2.wav
ExportingARI6-1634932302.467-out-3.wav
ExportingARI6-1634932302.467-out-4.wav
ExportingARI6-1634932302.467-out-5.wav
ExportingARI6-1634932302.467-out-6.wav
ExportingARI6-1634932302.467-out-7.wav
ExportingARI6-1634932302.467-out-8.wav
ExportingARI6-1634932302.467-out-9.wav
ExportingARI6-1634932302.467-out-10.wav
Audio File #  87
ExportingARI8-1635895230.17502-in-0.wav
ExportingARI8-1635895230.17502-in-1.wav
Audio File #  88
ExportingARI8-1635006257.4194-in-0.wav
Audio File #  89
ExportingARI9-1635002275.5855-in-0.wav
Audio File #  90
ExportingARI7-1635001491.1104-in-0.wav
Audio File #  91
ExportingARI9-1635958451.14333-in-0.wav
ExportingARI9-1635958451.14333-in-1.wav
Audio File #  92
ExportingARI6-1635618236.59514-in-0.wav
Audio File #  93
ExportingARI6-1635890003.1686-in-0.wav
ExportingARI6-1635890003.1686-in-1.wav
Audio File #  94
Audio File #  95
Expor

Audio File #  167
ExportingARI9-1635782887.27143-in-0.wav
ExportingARI9-1635782887.27143-in-1.wav
Audio File #  168
ExportingARI8-1635779370.1833-in-0.wav
ExportingARI8-1635779370.1833-in-1.wav
ExportingARI8-1635779370.1833-in-2.wav
ExportingARI8-1635779370.1833-in-3.wav
ExportingARI8-1635779370.1833-in-4.wav
ExportingARI8-1635779370.1833-in-5.wav
Audio File #  169
ExportingARI12-1635867255.1864-in-0.wav
ExportingARI12-1635867255.1864-in-1.wav
Audio File #  170
ExportingARI10-1635007357.25568-in-0.wav
ExportingARI10-1635007357.25568-in-1.wav
Audio File #  171
Audio File #  172
ExportingARI6-1635622953.84619-in-0.wav
ExportingARI6-1635622953.84619-in-1.wav
ExportingARI6-1635622953.84619-in-2.wav
Audio File #  173
ExportingARI12-1634923028.11209-in-0.wav
ExportingARI12-1634923028.11209-in-1.wav
Audio File #  174
ExportingARI9-1635779647.8641-in-0.wav
ExportingARI9-1635779647.8641-in-1.wav
ExportingARI9-1635779647.8641-in-2.wav
Audio File #  175
ExportingARI12-1635874038.31466-in-0.wav
Ex

Audio File #  244
ExportingARI5-1635787350.19206-in-0.wav
Audio File #  245
ExportingARI6-1635618398.60335-in-0.wav
Audio File #  246
ExportingARI10-1635952914.9242-in-0.wav
ExportingARI10-1635952914.9242-in-1.wav
Audio File #  247
ExportingARI5-1635785565.15047-in-0.wav
Audio File #  248
ExportingARI5-1635782409.5519-in-0.wav
ExportingARI5-1635782409.5519-in-1.wav
ExportingARI5-1635782409.5519-in-2.wav
Audio File #  249
ExportingARI8-1635893182.4379-in-0.wav
ExportingARI8-1635893182.4379-in-1.wav
ExportingARI8-1635893182.4379-in-2.wav
ExportingARI8-1635893182.4379-in-3.wav
ExportingARI8-1635893182.4379-in-4.wav
ExportingARI8-1635893182.4379-in-5.wav
ExportingARI8-1635893182.4379-in-6.wav
ExportingARI8-1635893182.4379-in-7.wav
ExportingARI8-1635893182.4379-in-8.wav
ExportingARI8-1635893182.4379-in-9.wav
ExportingARI8-1635893182.4379-in-10.wav
ExportingARI8-1635893182.4379-in-11.wav
Audio File #  250
ExportingARI5-1635802056.43645-in-0.wav
ExportingARI5-1635802056.43645-in-1.wav
Exporti

Audio File #  306
ExportingARI8-1635623386.97449-in-0.wav
ExportingARI8-1635623386.97449-in-1.wav
ExportingARI8-1635623386.97449-in-2.wav
Audio File #  307
ExportingARI8-1635782030.9057-in-0.wav
ExportingARI8-1635782030.9057-in-1.wav
ExportingARI8-1635782030.9057-in-2.wav
Audio File #  308
ExportingARI6-1635620256.69541-in-0.wav
ExportingARI6-1635620256.69541-in-1.wav
ExportingARI6-1635620256.69541-in-2.wav
Audio File #  309
ExportingARI9-1635952918.5716-in-0.wav
Audio File #  310
ExportingARI5-1635884429.214-in-0.wav
ExportingARI5-1635884429.214-in-1.wav
ExportingARI5-1635884429.214-in-2.wav
Audio File #  311
ExportingARI10-1634941273.49719-in-0.wav
Audio File #  312
ExportingARI8-1635797569.16837-in-0.wav
ExportingARI8-1635797569.16837-in-1.wav
Audio File #  313
ExportingARI8-1635799566.22685-in-0.wav
ExportingARI8-1635799566.22685-in-1.wav
Audio File #  314
ExportingARI12-1635883764.14-in-0.wav
ExportingARI12-1635883764.14-in-1.wav
Audio File #  315
ExportingARI8-1635895373.18509-in

...

Audio File #  1368
ExportingARI6-1635619212.64343-in-0.wav
Audio File #  1369
ExportingARI9-1635778302.335-in-0.wav
Audio File #  1370
ExportingARI12-1635872585.24674-in-0.wav
Audio File #  1371
ExportingARI10-1634933828.33813-in-0.wav
Audio File #  1372
ExportingARI10-1634926692.25385-in-0.wav
ExportingARI10-1634926692.25385-in-1.wav
Audio File #  1373
ExportingARI11-1635873470.12724-in-0.wav
Audio File #  1374
ExportingARI9-1634923863.9328-in-0.wav
Audio File #  1375
ExportingARI13-1635877728.3708-in-0.wav
Audio File #  1376
ExportingARI10-1635952434.8573-in-0.wav
Audio File #  1377
ExportingARI6-1635623328.86741-in-0.wav
ExportingARI6-1635623328.86741-in-1.wav
Audio File #  1378
ExportingARI10-1634932203.30128-in-0.wav
ExportingARI10-1634932203.30128-in-1.wav
ExportingARI10-1634932203.30128-in-2.wav
Audio File #  1379
ExportingARI6-1635619685.66644-in-0.wav
Audio File #  1380
ExportingARI12-1635872974.26584-in-0.wav
Audio File #  1381
ExportingARI8-1635622958.96293-in-0.wav
Exportin

Audio File #  1434
ExportingARI12-1635868666.8224-in-0.wav
ExportingARI12-1635868666.8224-in-1.wav
Audio File #  1435
ExportingARI8-1635799225.21686-in-0.wav
Audio File #  1436
ExportingARI5-1635798211.33442-in-0.wav
Audio File #  1437
ExportingARI12-1634923688.14189-in-0.wav
ExportingARI12-1634923688.14189-in-1.wav
Audio File #  1438
ExportingARI9-1635784201.32828-in-0.wav
Audio File #  1439
ExportingARI8-1635800242.24638-in-0.wav
ExportingARI8-1635800242.24638-in-1.wav
Audio File #  1440
ExportingARI6-1634935951.7019-in-0.wav
ExportingARI6-1634935951.7019-in-1.wav
Audio File #  1441
ExportingARI10-1635000034.18-in-0.wav
ExportingARI10-1635000034.18-in-1.wav
ExportingARI10-1635000034.18-in-2.wav
ExportingARI10-1635000034.18-in-3.wav
ExportingARI10-1635000034.18-in-4.wav
ExportingARI10-1635000034.18-in-5.wav
ExportingARI10-1635000034.18-in-6.wav
Audio File #  1442
ExportingARI7-1635893321.5398-in-0.wav
Audio File #  1443
ExportingARI5-1635885292.2993-in-0.wav
Audio File #  1444
Exporti

Audio File #  1520
ExportingARI8-1635618685.83007-in-0.wav
ExportingARI8-1635618685.83007-in-1.wav
Audio File #  1521
ExportingARI10-1635787593.29109-in-0.wav
Audio File #  1522
ExportingARI12-1634926288.25603-in-0.wav
Audio File #  1523
ExportingARI12-1635869716.12515-in-0.wav
Audio File #  1524
ExportingARI10-1635004045.14942-in-0.wav
Audio File #  1525
ExportingARI10-1635795968.35693-in-0.wav
ExportingARI10-1635795968.35693-in-1.wav
ExportingARI10-1635795968.35693-in-2.wav
ExportingARI10-1635795968.35693-in-3.wav
Audio File #  1526
ExportingARI11-1635868056.4983-in-0.wav
Audio File #  1527
ExportingARI12-1635872614.24826-in-0.wav
Audio File #  1528
ExportingARI8-1634932938.8779-in-0.wav
ExportingARI8-1634932938.8779-in-1.wav
Audio File #  1529
ExportingARI10-1635618282.177801-in-0.wav
Audio File #  1530
ExportingARI12-1635873532.29229-in-0.wav
ExportingARI12-1635873532.29229-in-1.wav
Audio File #  1531
ExportingARI9-1635951623.2359-in-0.wav
Audio File #  1532
ExportingARI5-163578216

Audio File #  1612
ExportingARI6-1635621347.75212-in-0.wav
Audio File #  1613
ExportingARI9-1634922266.4918-in-0.wav
Audio File #  1614
ExportingARI5-1635892804.21576-in-0.wav
ExportingARI5-1635892804.21576-in-1.wav
Audio File #  1615
ExportingARI8-1635007428.7479-in-0.wav
ExportingARI8-1635007428.7479-in-1.wav
ExportingARI8-1635007428.7479-in-2.wav
Audio File #  1616
ExportingARI6-1635619805.67239-in-0.wav
Audio File #  1617
ExportingARI12-1635866987.532-in-0.wav
ExportingARI12-1635866987.532-in-1.wav
Audio File #  1618
ExportingARI6-1635002476.5581-in-0.wav
ExportingARI6-1635002476.5581-in-1.wav
Audio File #  1619
ExportingARI10-1635625884.196851-in-0.wav
ExportingARI10-1635625884.196851-in-1.wav
ExportingARI10-1635625884.196851-in-2.wav
Audio File #  1620
ExportingARI7-1635951560.10533-in-0.wav
ExportingARI7-1635951560.10533-in-1.wav
ExportingARI7-1635951560.10533-in-2.wav
Audio File #  1621
ExportingARI5-1635886087.6903-in-0.wav
Audio File #  1622
ExportingARI12-1635866941.272-in-0

C:\Users\trainee6\Anaconda3\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


In [10]:
audio_file_name_list_2 = audio_file_name_list[1672:]

In [11]:
# previous chunks 4097

# import os

# file_names = os.listdir(r"Z:\audios\T_1")

try :
    
    for file_name in audio_file_name_list_2:
        # Load your audio.
        song = AudioSegment.from_wav("Z:/audios/T_1/" + str(file_name))

        # Split track where the silence is 2 seconds or more and get chunks using 
        # the imported function.
        chunks = split_on_silence (
            # Use the loaded audio.
            song, 
            # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
            min_silence_len = 1800, #1800
            # Consider a chunk silent if it's quieter than -16 dBFS.
            # (You may want to adjust this parameter.)
            silence_thresh = song.dBFS, #-27

            keep_silence=300,

            seek_step=1
        )

        file_name = file_name.replace('.wav', '')
        
        print('Audio File # ' ,count)
        count+=1
        
        # Process each chunk with your parameters
        for i, chunk in enumerate(chunks):
            # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
            silence_chunk = AudioSegment.silent(duration=500)

            # Add the padding chunk to beginning and end of the entire chunk.
            audio_chunk = silence_chunk + chunk + silence_chunk

            # Normalize the entire chunk.
            normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

            # Export the audio chunk with new bitrate.
            print("Exporting" + file_name + "-{0}.wav".format(i))
            normalized_chunk.export(
                "audio_chunks_T_1/" + file_name + "-{0}.wav".format(i+1),
                bitrate = "192k",
                format = "wav"
            )
            
            
except Exception as e:
        print(e)
            
            
# except FileNotFoundError:
#     pass

# except RuntimeWarning:
#     pass

Audio File #  1671
ExportingARI10-1635624622.193026-in-0.wav
Audio File #  1672
ExportingARI7-1635893273.5062-in-0.wav
ExportingARI7-1635893273.5062-in-1.wav
ExportingARI7-1635893273.5062-in-2.wav
Audio File #  1673
ExportingARI11-1635868005.4923-in-0.wav
Audio File #  1674
ExportingARI10-1635625058.194298-in-0.wav
Audio File #  1675
ExportingARI13-1635879054.8329-in-0.wav
Audio File #  1676
Audio File #  1677
ExportingARI9-1635781139.17132-in-0.wav
ExportingARI9-1635781139.17132-in-1.wav
Audio File #  1678
ExportingARI7-1635005612.9211-in-0.wav
Audio File #  1679
ExportingARI9-1635782099.22876-in-0.wav
ExportingARI9-1635782099.22876-in-1.wav
Audio File #  1680
ExportingARI12-1635873319.28267-in-0.wav
ExportingARI12-1635873319.28267-in-1.wav
Audio File #  1681
ExportingARI11-1635871761.10103-in-0.wav
ExportingARI11-1635871761.10103-in-1.wav
Audio File #  1682
ExportingARI10-1635620910.183475-in-0.wav
ExportingARI10-1635620910.183475-in-1.wav
ExportingARI10-1635620910.183475-in-2.wav
Ex

Audio File #  1748
ExportingARI11-1635870319.7966-in-0.wav
ExportingARI11-1635870319.7966-in-1.wav
Audio File #  1749
ExportingARI6-1634937445.13502-out-0.wav
ExportingARI6-1634937445.13502-out-1.wav
Audio File #  1750
ExportingARI7-1635892920.2925-in-0.wav
ExportingARI7-1635892920.2925-in-1.wav
Audio File #  1751
ExportingARI5-1635795696.26259-in-0.wav
ExportingARI5-1635795696.26259-in-1.wav
Audio File #  1752
ExportingARI9-1635782438.24919-in-0.wav
Audio File #  1753
ExportingARI11-1635866723.1678-in-0.wav
Audio File #  1754
ExportingARI8-1634938008.25095-out-0.wav
ExportingARI8-1634938008.25095-out-1.wav
Audio File #  1755
ExportingARI12-1634924254.16765-in-0.wav
ExportingARI12-1634924254.16765-in-1.wav
Audio File #  1756
ExportingARI6-1634944545.34566-out-0.wav
ExportingARI6-1634944545.34566-out-1.wav
Audio File #  1757
ExportingARI9-1635780580.13589-in-0.wav
ExportingARI9-1635780580.13589-in-1.wav
ExportingARI9-1635780580.13589-in-2.wav
ExportingARI9-1635780580.13589-in-3.wav
Audi

Audio File #  1820
ExportingARI11-1635868476.5558-in-0.wav
Audio File #  1821
ExportingARI12-1635869875.13075-in-0.wav
ExportingARI12-1635869875.13075-in-1.wav
Audio File #  1822
ExportingARI5-1635891898.19171-in-0.wav
Audio File #  1823
ExportingARI7-1635892502.623-in-0.wav
Audio File #  1824
ExportingARI10-1634925798.22845-in-0.wav
ExportingARI10-1634925798.22845-in-1.wav
Audio File #  1825
ExportingARI13-1635878359.5867-in-0.wav
ExportingARI13-1635878359.5867-in-1.wav
ExportingARI13-1635878359.5867-in-2.wav
ExportingARI13-1635878359.5867-in-3.wav
ExportingARI13-1635878359.5867-in-4.wav
Audio File #  1826
ExportingARI10-1635003206.11112-in-0.wav
Audio File #  1827
ExportingARI8-1635005993.3418-in-0.wav
Audio File #  1828
ExportingARI8-1635893285.5063-in-0.wav
Audio File #  1829
ExportingARI11-1635883900.804-in-0.wav
ExportingARI11-1635883900.804-in-1.wav
Audio File #  1830
ExportingARI12-1634924310.17044-in-0.wav
Audio File #  1831
ExportingARI7-1635626227.11481-in-0.wav
ExportingARI

Audio File #  1902
ExportingARI5-1635886383.8301-in-0.wav
Audio File #  1903
ExportingARI8-1634942082.39824-out-0.wav
ExportingARI8-1634942082.39824-out-1.wav
ExportingARI8-1634942082.39824-out-2.wav
ExportingARI8-1634942082.39824-out-3.wav
ExportingARI8-1634942082.39824-out-4.wav
ExportingARI8-1634942082.39824-out-5.wav
ExportingARI8-1634942082.39824-out-6.wav
ExportingARI8-1634942082.39824-out-7.wav
ExportingARI8-1634942082.39824-out-8.wav
ExportingARI8-1634942082.39824-out-9.wav
ExportingARI8-1634942082.39824-out-10.wav
ExportingARI8-1634942082.39824-out-11.wav
ExportingARI8-1634942082.39824-out-12.wav
Audio File #  1904
ExportingARI10-1635887864.6189-in-0.wav
Audio File #  1905
ExportingARI10-1634941133.49424-in-0.wav
Audio File #  1906
ExportingARI8-1635626423.106112-in-0.wav
Audio File #  1907
ExportingARI6-1635003521.9406-in-0.wav
ExportingARI6-1635003521.9406-in-1.wav
Audio File #  1908
ExportingARI6-1635619716.66809-in-0.wav
Audio File #  1909
ExportingARI5-1635885765.5317-in-

Audio File #  1977
Audio File #  1978
ExportingARI6-1635618175.59249-in-0.wav
Audio File #  1979
ExportingARI10-1635781590.9627-in-0.wav
ExportingARI10-1635781590.9627-in-1.wav
Audio File #  1980
ExportingARI13-1634934863.2026-out-0.wav
ExportingARI13-1634934863.2026-out-1.wav
ExportingARI13-1634934863.2026-out-2.wav
ExportingARI13-1634934863.2026-out-3.wav
ExportingARI13-1634934863.2026-out-4.wav
ExportingARI13-1634934863.2026-out-5.wav
Audio File #  1981
ExportingARI10-1634921533.3797-out-0.wav
ExportingARI10-1634921533.3797-out-1.wav
ExportingARI10-1634921533.3797-out-2.wav
ExportingARI10-1634921533.3797-out-3.wav
ExportingARI10-1634921533.3797-out-4.wav
ExportingARI10-1634921533.3797-out-5.wav
ExportingARI10-1634921533.3797-out-6.wav
Audio File #  1982
ExportingARI9-1634939463.42097-out-0.wav
ExportingARI9-1634939463.42097-out-1.wav
ExportingARI9-1634939463.42097-out-2.wav
ExportingARI9-1634939463.42097-out-3.wav
ExportingARI9-1634939463.42097-out-4.wav
Audio File #  1983
Exporting

...

Audio File #  4952
ExportingARI12-1634925742.23706-in-0.wav
Audio File #  4953
ExportingARI6-1634939261.20864-out-0.wav
ExportingARI6-1634939261.20864-out-1.wav
ExportingARI6-1634939261.20864-out-2.wav
ExportingARI6-1634939261.20864-out-3.wav
ExportingARI6-1634939261.20864-out-4.wav
ExportingARI6-1634939261.20864-out-5.wav
Audio File #  4954
ExportingARI6-1634932616.2091-in-0.wav
Audio File #  4955
ExportingARI8-1635796114.12729-in-0.wav
ExportingARI8-1635796114.12729-in-1.wav
Audio File #  4956
ExportingARI12-1634922166.7156-out-0.wav
ExportingARI12-1634922166.7156-out-1.wav
ExportingARI12-1634922166.7156-out-2.wav
ExportingARI12-1634922166.7156-out-3.wav
ExportingARI12-1634922166.7156-out-4.wav
ExportingARI12-1634922166.7156-out-5.wav
Audio File #  4957
ExportingARI9-1634923457.8025-in-0.wav
ExportingARI9-1634923457.8025-in-1.wav
Audio File #  4958
ExportingARI5-1635788412.21817-in-0.wav
ExportingARI5-1635788412.21817-in-1.wav
Audio File #  4959
ExportingARI10-1634941242.49655-in-0.w

Audio File #  5024
ExportingARI5-1635894056.25025-in-0.wav
ExportingARI5-1635894056.25025-in-1.wav
Audio File #  5025
ExportingARI10-1635785956.24129-in-0.wav
ExportingARI10-1635785956.24129-in-1.wav
ExportingARI10-1635785956.24129-in-2.wav
ExportingARI10-1635785956.24129-in-3.wav
Audio File #  5026
ExportingARI10-1635003892.14403-in-0.wav
ExportingARI10-1635003892.14403-in-1.wav
ExportingARI10-1635003892.14403-in-2.wav
ExportingARI10-1635003892.14403-in-3.wav
Audio File #  5027
ExportingARI11-1635870601.8339-in-0.wav
Audio File #  5028
ExportingARI7-1635951151.8912-in-0.wav
Audio File #  5029
ExportingARI10-1634937985.42764-in-0.wav
ExportingARI10-1634937985.42764-in-1.wav
ExportingARI10-1634937985.42764-in-2.wav
ExportingARI10-1634937985.42764-in-3.wav
Audio File #  5030
ExportingARI13-1635878393.5995-in-0.wav
ExportingARI13-1635878393.5995-in-1.wav
ExportingARI13-1635878393.5995-in-2.wav
ExportingARI13-1635878393.5995-in-3.wav
ExportingARI13-1635878393.5995-in-4.wav
Audio File #  50

Audio File #  5088
ExportingARI7-1635953845.17579-in-0.wav
Audio File #  5089
ExportingARI10-1635623719.190489-in-0.wav
ExportingARI10-1635623719.190489-in-1.wav
Audio File #  5090
ExportingARI7-1635002463.3798-in-0.wav
Audio File #  5091
ExportingARI8-1635894541.13231-in-0.wav
Audio File #  5092
ExportingARI12-1635868996.9541-in-0.wav
ExportingARI12-1635868996.9541-in-1.wav
ExportingARI12-1635868996.9541-in-2.wav
Audio File #  5093
ExportingARI9-1634921216.1737-in-0.wav
ExportingARI9-1634921216.1737-in-1.wav
Audio File #  5094
ExportingARI8-1635619940.91059-in-0.wav
ExportingARI8-1635619940.91059-in-1.wav
ExportingARI8-1635619940.91059-in-2.wav
ExportingARI8-1635619940.91059-in-3.wav
ExportingARI8-1635619940.91059-in-4.wav
Audio File #  5095
ExportingARI6-1635001330.1252-in-0.wav
ExportingARI6-1635001330.1252-in-1.wav
Audio File #  5096
ExportingARI6-1635620838.72578-in-0.wav
ExportingARI6-1635620838.72578-in-1.wav
ExportingARI6-1635620838.72578-in-2.wav
Audio File #  5097
ExportingAR

Audio File #  5159
ExportingARI6-1635003661.9859-in-0.wav
ExportingARI6-1635003661.9859-in-1.wav
ExportingARI6-1635003661.9859-in-2.wav
Audio File #  5160
ExportingARI6-1635621814.77811-in-0.wav
ExportingARI6-1635621814.77811-in-1.wav
Audio File #  5161
ExportingARI12-1635869855.13012-in-0.wav
ExportingARI12-1635869855.13012-in-1.wav
Audio File #  5162
ExportingARI9-1634921712.3236-in-0.wav
Audio File #  5163
ExportingARI6-1634933189.3472-in-0.wav
ExportingARI6-1634933189.3472-in-1.wav
Audio File #  5164
ExportingARI12-1634922811.10126-in-0.wav
Audio File #  5165
ExportingARI10-1635782435.12639-in-0.wav
ExportingARI10-1635782435.12639-in-1.wav
Audio File #  5166
ExportingARI7-1635625874.10396-in-0.wav
Audio File #  5167
ExportingARI6-1635952871.10661-in-0.wav
ExportingARI6-1635952871.10661-in-1.wav
Audio File #  5168
ExportingARI11-1635883765.18-in-0.wav
Audio File #  5169
ExportingARI8-1635797113.15622-in-0.wav
ExportingARI8-1635797113.15622-in-1.wav
Audio File #  5170
ExportingARI9-1

Audio File #  5232
ExportingARI8-1635894914.15575-in-0.wav
ExportingARI8-1635894914.15575-in-1.wav
ExportingARI8-1635894914.15575-in-2.wav
ExportingARI8-1635894914.15575-in-3.wav
ExportingARI8-1635894914.15575-in-4.wav
ExportingARI8-1635894914.15575-in-5.wav
ExportingARI8-1635894914.15575-in-6.wav
ExportingARI8-1635894914.15575-in-7.wav
ExportingARI8-1635894914.15575-in-8.wav
ExportingARI8-1635894914.15575-in-9.wav
Audio File #  5233
ExportingARI13-1635885028.12639-in-0.wav
Audio File #  5234
ExportingARI7-1635953997.18026-in-0.wav
Audio File #  5235
ExportingARI9-1635780470.12875-in-0.wav
Audio File #  5236
ExportingARI10-1634932611.31015-out-0.wav
Audio File #  5237
ExportingARI5-1635891377.17636-in-0.wav
ExportingARI5-1635891377.17636-in-1.wav
Audio File #  5238
ExportingARI10-1635006878.23934-in-0.wav
Audio File #  5239
ExportingARI9-1635782856.26985-in-0.wav
Audio File #  5240
ExportingARI6-1635618426.60485-in-0.wav
Audio File #  5241
ExportingARI6-1634936518.9374-in-0.wav
Audio F

In [ ]:
# 5262

# audio_file_name_list_3 = audio_file_name_list[1672:]

audio_file_name_list[5263]

In [ ]:
# created_audio_chunks_path = os.listdir(r"C:\Users\trainee6\Desktop\Abdullah\audio_chunks")

In [ ]:
# len(created_audio_chunks_path)

In [ ]:
# created_audio_chunks_path

In [ ]:
# created_audio_chunks_path.to_csv('Abdullah_audio_chunks_path.csv', index=False)

In [39]:
# # file_names[16042]   'ARI8-1634939928.31945-in.wav'  yahan tk chunks bny hein

# file_names[16043:]

'ARI10-1635003972.14685-in.wav'

In [31]:
# file_names = list(file_names)

In [ ]:
#  Maam's code

import os
from pydub import AudioSegment
from pydub.silence import split_on_silence



# name_list = os.listdir('C:/Users/trainee9/Desktop/Sohaib/T_1') 
source ='//192.168.88.18/oddtech/audios/T_1/'
name_list = os.listdir(source)
count = 0
main_dir = "C:/Users/trainee9/Desktop/Sohaib/New_files2/"

for name in name_list:
    try:
        count = count +1
        sound_file = AudioSegment.from_wav(os.path.join(source ,str(name)))
        audio_chunks = split_on_silence(sound_file, min_silence_len=1900, silence_thresh=-35,  keep_silence=20 )
        name = name.replace('.wav','')
        for i, chunk in enumerate(audio_chunks):
            out_file =os.path.join(main_dir, str(count)+name+"-{0}.wav".format(i+1))
            print("exporting", out_file)
            if not os.path.exists(main_dir):
                os.mkdir(main_dir)
            chunk.export(out_file, format="wav")
        if count%10000==0:
            print("*"*30)
            print(count)
    except Exception as e:
        print(e)

In [ ]:
# import os
# from pydub import AudioSegment
# from pydub.silence import split_on_silence



# # name_list = os.listdir('C:/Users/trainee9/Desktop/Sohaib/T_1') 
# source ='//192.168.88.18/oddtech/audios/T_1/'
# name_list = os.listdir(source)
# count = 5263
# main_dir = "C:/Users/trainee6/Desktop/Abdullah/audio_chunks_T_1/"

# name_list = name_list[5263]

# for name in name_list:
#     try:
#         count = count +1
#         sound_file = AudioSegment.from_wav(os.path.join(source ,str(name)))
#         audio_chunks = split_on_silence(sound_file, min_silence_len=1900, silence_thresh=-35,  keep_silence=20 )
#         name = name.replace('.wav','')
#         for i, chunk in enumerate(audio_chunks):
#             out_file =os.path.join(main_dir, str(count)+name+"-{0}.wav".format(i+1))
#             print("exporting", out_file)
#             if not os.path.exists(main_dir):
#                 os.mkdir(main_dir)
#             chunk.export(out_file, format="wav")
#         if count%10000==0:
#             print("*"*30)
#             print(count)
#     except Exception as e:
#         print(e)